In [1]:
from bbox.metrics import iou_3d
from kiss_icp.tools.utils_class import BoundingBox3D
from typing import List
from scipy.spatial.transform import Rotation as R
import numpy as np
import pandas as pd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# different id because using filter
annotations_and_detections = np.load("../results/annotations_and_detections/000/000003.npy", allow_pickle='TRUE').item()

In [3]:
def compute_translation_error(p0, p1):
    dist = np.linalg.norm(p0 - p1)
    return dist

In [4]:
def compute_orientation_error(anno_R, detected_R):
    if anno_R[0][0] < 0 and anno_R[1][1] < 0:
        anno_R[0][0] = -anno_R[0][0] 
        anno_R[0][1] = -anno_R[0][1] 
        anno_R[1][0] = -anno_R[1][0] 
        anno_R[1][1] = -anno_R[1][1] 
    
    if detected_R[0][0] < 0 and detected_R[1][1] < 0:
        detected_R[0][0] = -detected_R[0][0] 
        detected_R[0][1] = -detected_R[0][1] 
        detected_R[1][0] = -detected_R[1][0] 
        detected_R[1][1] = -detected_R[1][1] 
        
    anno_r = R.from_matrix(anno_R)
    detected_r = R.from_matrix(detected_R)
    anno_euler = anno_r.as_euler('zxy', degrees=True)
    detected_euler = detected_r.as_euler('zxy', degrees=True)
    anno_yaw = anno_euler[0]
    detected_yaw = detected_euler[0]
    diff = np.abs(anno_yaw -  detected_yaw)
    
    # print("detected_R", detected_R)
    # print("anno_R", anno_R)
    
    # print("detected_euler", detected_euler)
    # print("anno_euler", anno_euler)

    # print("detected_yaw", detected_yaw)
    # print("anno_yaw", anno_yaw)
    # print("diff", diff)
    return diff

In [5]:
# print("annotations_and_detections\n", annotations_and_detections)

evaluation = {}
# for key, value in my_dict.items():
# for frame_id in range(len(annotations_and_detections)):
for frame_id in annotations_and_detections:
    frame = annotations_and_detections[frame_id]

    for track_uuid, anno_and_det in frame.items():

        try:
            if evaluation[track_uuid]:
                pass
        except:
            evaluation[track_uuid] = {}
        # if not evaluation[track_uuid]:
        #     evaluation[track_uuid] = {}
            
        evaluation[track_uuid][frame_id] = {}
        detection = anno_and_det[0]
        annotation = anno_and_det[1]

        annotation_bbox = BoundingBox3D(annotation.dst_SE3_object.translation[0], annotation.dst_SE3_object.translation[1],
        annotation.dst_SE3_object.translation[2], annotation.length_m, 
        annotation.width_m, annotation.height_m,
        annotation.dst_SE3_object.rotation)

        detected_bbox = detection.s_pose_ious[frame_id]
        detected_bbox_visuals = detection.s_pose_visuals[frame_id]
        evaluation[track_uuid][frame_id]["iou_3d"] = {}
        evaluation[track_uuid][frame_id]["instance_id"] = {}
        evaluation[track_uuid][frame_id]["translation_error"] = {}
        evaluation[track_uuid][frame_id]["orientation_error"] = {}
        evaluation[track_uuid][frame_id]["iou_3d"] = iou_3d(annotation_bbox.iou, detected_bbox)
        evaluation[track_uuid][frame_id]["instance_id"] = detection.id
        center_annotation = np.array([annotation.dst_SE3_object.translation[0], annotation.dst_SE3_object.translation[1],
        annotation.dst_SE3_object.translation[2]])
        
        center_detected_bbox_visuals = np.array([detected_bbox_visuals.x, detected_bbox_visuals.y, detected_bbox_visuals.z])
        evaluation[track_uuid][frame_id]["translation_error"] = compute_translation_error(center_annotation, center_detected_bbox_visuals)
        
        evaluation[track_uuid][frame_id]["orientation_error"] = compute_orientation_error(annotation.dst_SE3_object.rotation, 
                                                                                           detected_bbox_visuals.rot)

        

    # if frame_id == 1:
    #     break
    
# print("evaluation\n", evaluation)

In [6]:
# https://stackoverflow.com/questions/13575090/construct-pandas-dataframe-from-items-in-nested-dictionary
df = pd.DataFrame.from_records(
    [
        (track_uuid, frame_id, attributes['iou_3d'], attributes['instance_id']
         , attributes['translation_error'], attributes['orientation_error'])
        for track_uuid, info in evaluation.items()
        for frame_id, attributes in info.items()
    ],
    columns=['track_uuid', 'frame_id', 'iou_3d', 'instance_id', 'translation_error', 'orientation_error']
)
df

,track_uuid,frame_id,iou_3d,instance_id,translation_error,orientation_error
0,695efdce-a16d-4381-8edc-7797207255ae,0,0.67101,0,0.100234,173.506514
1,695efdce-a16d-4381-8edc-7797207255ae,1,0.63805,0,0.118267,177.197346
2,695efdce-a16d-4381-8edc-7797207255ae,2,0.67775,0,0.126602,178.623351
3,695efdce-a16d-4381-8edc-7797207255ae,3,0.76074,0,0.095035,175.311066
4,695efdce-a16d-4381-8edc-7797207255ae,4,0.62546,0,0.140369,176.918391
...,...,...,...,...,...,...
491,b7d674fe-01e5-4a14-89c7-5bc901dc2581,151,0.63813,188,0.485365,5.848336
492,b7d674fe-01e5-4a14-89c7-5bc901dc2581,152,0.76995,188,0.171368,4.299370
493,b7d674fe-01e5-4a14-89c7-5bc901dc2581,153,0.77520,188,0.169546,1.424066
494,b7d674fe-01e5-4a14-89c7-5bc901dc2581,154,0.83205,188,0.100891,2.796932


In [7]:
def get_results(df, track_uuids):
    '''
    Sum all iou_3d and divide by the number of instance_id
    '''
    for track_uuid in track_uuids:
        df_intance_iou_3d = df.loc[df['track_uuid'] == track_uuid, ['instance_id', 'iou_3d', 'translation_error', 'orientation_error']]
        df_intance_iou_3d_mean = df_intance_iou_3d.groupby("instance_id").mean()
        df_intance_iou_3d_count = df_intance_iou_3d.groupby("instance_id").count()
        merge = df_intance_iou_3d_mean.merge(df_intance_iou_3d_count, left_on='instance_id', right_on='instance_id')
        merge = merge.rename(columns={"iou_3d_x": "iou_3d", "translation_error_x": "translation_error",
                                     "orientation_error_x": "orientation_error", "iou_3d_y": "number_of_frames"})
        merge = merge[['iou_3d', 'translation_error', 'orientation_error', 'number_of_frames']]
        print('\n')
        print('Track_UUID', track_uuid)
        print(merge)

In [8]:
def show_interesting_track_uuids(df, minimum_frame_number=30):
    df1 = df.groupby(['track_uuid']).nunique()
    df2 = df1.loc[(df1['frame_id'] > minimum_frame_number), ['frame_id', 'instance_id', 'translation_error', 'orientation_error']]
    return df2

In [9]:
def get_track_uuid(df):
    df3 = df2['frame_id'].keys().to_list()
    return df3

In [10]:
# Important 
df2 = show_interesting_track_uuids(df, 0)
print(df2[['frame_id', 'instance_id']]) # num frame and num instance
track_uuids = get_track_uuid(df)
print(get_results(df, track_uuids))

                                      frame_id  instance_id
track_uuid                                                 
098b7745-676c-4816-a01d-0581eca4be64        45            3
1458bc97-c005-47b4-9ca4-74def0777ede        27            1
293fe4b1-c557-4d2f-a214-9308a50f153b        63            1
47b2b55c-51b7-46ae-b573-7d7016179b70        30            1
62dd6224-e191-42b6-8dbe-43dc3201c2e3        39            8
695efdce-a16d-4381-8edc-7797207255ae         7            1
6a2a0950-625c-44b1-a3a9-976f039dff37         4            1
798ce97a-8e85-4fca-94e1-777c76568578         5            1
8c4b7f04-6ea8-461b-ad46-044271cbb380        11            2
a439e659-2fe7-4b66-999d-76a756a119fb        15            1
ab43af66-f024-4b97-80fc-b1c7ca0d0288        98            7
b7d674fe-01e5-4a14-89c7-5bc901dc2581         9            1
c6660ef1-a241-4277-a09b-e5235080bda6         2            1
c8fe342b-de36-4e3a-a4d1-63e4958c6870         1            1
c9e8b5b3-a64a-4f00-9818-5feccb31544e    

In [11]:
# Tell overall value
# print(df.nunique())
# print(df.head())

# print(df.groupby(['track_uuid']).sum(numeric_only=True))
# print(df.keys())

# print(.keys())
# mylist = np.unique(df2.loc[:, :]['track_uuid'].values).tolist()
# print(df2.loc[:, :].keys())

# print(df.groupby(['track_uuid', 'instance_id']).nunique())
# df.keys() = Index(['track_uuid', 'frame_id', 'iou_3d', 'instance_id'], dtype='object')